In [87]:
import requests
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time # selenium을 사용할 때 time 반드시 import

In [ ]:
url = 'http://consensus.hankyung.com/apps.analysis/analysis.list?&sdate=2018-10-22&edate=2021-10-22&report_type=IN&order_type=&now_page=1'
req = requests.get(url)
html = req.text

In [89]:
driver = webdriver.Chrome('/Users/Hwangmi/Downloads/chromedriver')
driver.get(url)

In [90]:
# BeautifulSoup 이용
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [91]:
divs = soup.select('div.layerPop') # 복수 개의 요소 찾기
len(divs)

20

In [94]:
div = divs[0]
title = div.select_one('.pop01.disNone').text # 하나의 요소 찾기
title = ' '.join(i for i in title.strip().split('\n'))
title

'자동차 - Tesla 3Q21 리뷰: 이어지는 놀라운 실적  3Q21 실적 서프라이즈: EPS $1.86 기록 (Non-GAAP 기준)Berlin, Austin Gigafactory 연내가동 확정, 당분간 가파른 판매증가세 기대'

In [95]:
soup.select('td.first.txt_number')[0].text

'2021-10-22'

In [96]:
from tqdm import tqdm
title_list = []
date_list = []

for page in tqdm(range(1,1028)):
    url = f'http://consensus.hankyung.com/apps.analysis/analysis.list?&sdate=2018-10-22&edate=2021-10-22&report_type=IN&order_type=&now_page={page}'
    driver.get(url)
    time.sleep(3) # for문을 사용하기 때문에 반드시 사용하기, page를 읽어올 수 있는 충분한 시간 주기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    divs = soup.select('div.layerPop') # 복수 개의 요소 찾기
    dates = soup.select('td.first.txt_number')
    for k in range(len(divs)):
        div = divs[k]
        title = div.select_one('.pop01.disNone').text # 하나의 요소 찾기
        title_list.append(' '.join(i for i in title.strip().split('\n')))
        date_list.append(dates[k].text)

100%|██████████| 1027/1027 [1:25:28<00:00,  4.99s/it]


In [97]:
title_list[:5]

['자동차 - Tesla 3Q21 리뷰: 이어지는 놀라운 실적  3Q21 실적 서프라이즈: EPS $1.86 기록 (Non-GAAP 기준)Berlin, Austin Gigafactory 연내가동 확정, 당분간 가파른 판매증가세 기대',
 '자동차 - 수소차 정부, 2030년 수소차 88만대 보급 목표  2030 년 수소차 보급 목표 88 만대 발표수소차/ 전기차 부문 탄소감축 목표 달성에 정책 지원 우선 예상',
 "항공 - Space Industry 한국형 우주 발사체 누리호, 미완의 성공  한국형 우주 발사체 '누리호' 발사 부분 성공발사 기술력은 검증 완료",
 '통신 - 3Q21 Preview : 넘버스에 더해진 내러티브  통신서비스 3사 합산 영업이익은 컨센서스를 2% 상회 전망마케팅비용 자산 상각 규모 축소 영향이 나타난 LG유플러스 실적 양호',
 '조선 - meritz 조선/기계 Daily  국내외 주요 뉴스 및 코멘트']

In [98]:
date_list[-5:]

['2018-10-22', '2018-10-22', '2018-10-22', '2018-10-22', '2018-10-22']

In [104]:
len(title_list), len(date_list)

(20535, 20535)

In [99]:
driver.close()

In [131]:
news = pd.DataFrame({'date':date_list, 'title': title_list})
news.head()

,date,title
0,2021-10-22,자동차 - Tesla 3Q21 리뷰: 이어지는 놀라운 실적 3Q21 실적 서프라이...
1,2021-10-22,"자동차 - 수소차 정부, 2030년 수소차 88만대 보급 목표 2030 년 수소차..."
2,2021-10-22,"항공 - Space Industry 한국형 우주 발사체 누리호, 미완의 성공 한국..."
3,2021-10-22,통신 - 3Q21 Preview : 넘버스에 더해진 내러티브 통신서비스 3사 합산...
4,2021-10-22,조선 - meritz 조선/기계 Daily 국내외 주요 뉴스 및 코멘트


In [128]:
news.to_csv('hankyung_news.csv', index=False)